# SOLID Design Principles

Useful principles of object-oriented design. Design patterns are reusable solution to common programming problems. It is designed and Introduced by Robert C. Martin.

**SOLID stands for:**
- S - Single Responsibility Principle
- O - Open-Closed Principle
- L - Liskov Substitution Principle
- I - Interface Segregation Principle
- D - Dependency Inversion Principle

## Single Responsibility Principle _or_ Separation of Concern

When you have a class, the class should have it's primary responsibility and it should not take on other responsibility. 

In [1]:
# SRP SOC

class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0
        
    def add_entry(self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
        
    def remove_entry(self, position):
        del self.entries[position]
        
    def __str__(self):
        return "\n".join(self.entries)

In [2]:
journal = Journal()
journal.add_entry("Beautiful day!")
journal.add_entry("Bad Weather")

In [3]:
journal.count

2

In [4]:
str(journal)

'1: Beautiful day!\n2: Bad Weather'

So far, it is following the SRP (single responsibility principle)
Now, we'll try to add some more functionality to it.

In [5]:
# SRP SOC

class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0
        
    def add_entry(self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
        
    def remove_entry(self, position):
        del self.entries[position]
        
    def save(self, filename):
        with open(filename, 'w') as f:
            f.write(str(self))
            
    def load(self):
        pass
        
    def load_from_web(self):
        pass
        
    def __str__(self):
        return "\n".join(self.entries)

Now, the problem with above class is that we've added secondary responsibility.  
It it is taking the responsibility of persistance. Which are our `save`, `load` & `load_from_web` method.

In order to manage all of that, we should be using another class to manage persistency.

In [6]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0
        
    def add_entry(self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
        
    def remove_entry(self, position):
        del self.entries[position]
        
    def __str__(self):
        return "\n".join(self.entries)
    

class PersistenceManager:
    
    @staticmethod
    def save_to_file(journal, filename):
        with open(filename, 'w') as f:
            f.write(str(journal))

In [7]:
j = Journal()
j.add_entry("I cried today")
j.add_entry("I ate a bug")
PersistenceManager().save_to_file(j, "journal.txt")

In [8]:
open("journal.txt").read()

'1: I cried today\n2: I ate a bug'

_PS: Single Responsibility Principle helps us to prevent from creating god object, means, it has alot of functionality which is anti-pattern_

## Open Closed Principle

OCP means - Open for extension, Closed for modification

In [9]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
    
class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3

In [10]:
class Product:
    def __init__(self, name, color, size):
        self.name = name
        self.color = color
        self.size = size

In [11]:
class ProductFilter:
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color:
                yield p
                
    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size:
                yield p
                
    def filter_by_size_and_color(self, products, size, color):
        for p in products:
            if p.size == size and p.color == color:
                yield p

In above class `ProductFilter` you can see that based on certain conditions we are adding filters, which is not a good design, it can't be scaled also with new requirements we've to keep adding new functions which is not adviced.

Imagine you've 3 or 4 criteria, then this class will clearly explode, you will end up writing multiple functions.

So, in order to fix this issue, we are going to create something called as Enterprise pattern (Specification) which will help us to deal with this problem.

In [12]:
class Specification:
    def is_satisfied(self, item):
        pass
    
class Filter:
    def filter(self, items, spec: Specification):
        pass
    

Now, let's say you want to filter by color, Then you'll create a color specification class

In [13]:
class ColorSpecification(Specification):
    def __init__(self, color: Color):
        self.color = color
    
    def is_satisfied(self, item):
        return item.color == self.color

Similarly, we are going to create specification for other filters

In [14]:
class SizeSpecification(Specification):
    def __init__(self, size: Size):
        self.size = size
        
    def is_satisfied(self, item):
        return item.size == self.size

Now, we'll see how we can actually use this custom specification classes, in order to do that, we'll also inherit from the filter class.

In [24]:
class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item

In [16]:
apple = Product("Apple", Color.GREEN, Size.SMALL)
tree = Product("Tree", Color.GREEN, Size.LARGE)
house = Product("House", Color.BLUE, Size.LARGE)

products = [apple, tree, house]

In [21]:
## Filtering using old approach
pf = ProductFilter()
print("Green Products (old)")
for p in pf.filter_by_color(products, Color.GREEN):
    print(p.name)

Green Products (old)
Apple
Tree


In [26]:
## Filtering using new approach

# better filter instance
bf = BetterFilter()

# creating color specification
green = ColorSpecification(Color.GREEN)

print("Green Products (new)")
for p in bf.filter(products, green):
    print(p.name)
    
print("Large size products (new)")
large = SizeSpecification(Size.LARGE)
for p in bf.filter(products, large):
    print(p.name)

Green Products (new)
Apple
Tree
Large size products (new)
Tree
House


Now, the question is how can we implement multiple specification at the same time, eg: size and color of certain type. In order to do that, we need something called as combinator

In [27]:
class AndSpecification(Specification):
    def __init__(self, *args):
        self.args = args
        
    def is_satisfied(self, item):
        return all(map(
            lambda spec: spec.is_satisfied(item), self.args
        ))

In [28]:
print("Large blue items")
large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))

for p in bf.filter(products, large_blue):
    print(p.name)

Large blue items
House


This is how we can add multiple and filters to the set of products

Another way to deal with it in more intutive way is to override `&` operator

In [31]:
class Specification:
    def is_satisfied(self, item):
        pass
    
    def __and__(self, other):
        return AndSpecification(self, other)

In [33]:
class ColorSpecification(Specification):
    def __init__(self, color: Color):
        self.color = color
    
    def is_satisfied(self, item):
        return item.color == self.color
    
class SizeSpecification(Specification):
    def __init__(self, size: Size):
        self.size = size
        
    def is_satisfied(self, item):
        return item.size == self.size



size_spec = SizeSpecification(Size.SMALL)
color_spec = ColorSpecification(Color.GREEN)

small_green = size_spec & color_spec

for p in bf.filter(products, small_green):
    print(p.name)

Apple


## Liskov Substitution Principle

The idea is if you have some interface that take some sort of base class, you should be able to stick a derived class in there and everything should work.

In [37]:
class Rectangle:
    def __init__(self, width, height):
        self._height = height
        self._width = width
        
    @property
    def area(self):
        return self._width * self._height
    
    def __str__(self):
        return f"Width: {self.width}, Height: {self.height}"
        
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        self._width = value
        
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        self._height = value

Why did we do it?
Well we'll see a particular kind of side effect of breaking the risk of substitution principle using inheritance

In [38]:
def use_it(rc):
    w = rc.width
    rc.height = 10
    expected = int(w*10)
    print(f"Expected an area of {expected}, got {rc.area}")

In [39]:
rc = Rectangle(2, 3)
use_it(rc)

Expected an area of 20, got 20


Now, let see how we can break the list of substitution principle by making a derived class, which inherits from Rectangle which absolutely does not work with this method.

And, in this process we'll see why we chose to use properties as opposed to just attributes

In [46]:
class Square(Rectangle):
    def __init__(self, size):
        Rectangle.__init__(self, size, size)
        
    @Rectangle.width.setter
    def width(self, value):
        self._width = self._height = value
        
    @Rectangle.height.setter
    def height(self, value):
        self._width = self._height = value

Unfortunately, this breaks the Liskov Substitution principle. 

In [47]:
sq = Square(5)
use_it(sq)

Expected an area of 50, got 100


As we can see above, the answer is incorrect, so what is the problem here.

```
def use_it(rc):
    w = rc.width
    rc.height = 10 #### This is the problem, rc.height = 10, has a side effect which will also change the width, so the width that we got here is no longer valid
    expected = int(w*10)
    print(f"Expected an area of {expected}, got {rc.area}")
```

## Interface Segregation Principle

The idea is that you don't really want to stick to many elements, to many methods into an interface, So let's suppose that you are trying to define some sort of machine for printing and scanning and faxing things and so on. So it might seem like a good idea to just define a single interface that's a rather large interface and then let you clients kind of implement this however they want

In [1]:
class Machine:
    def print(self, document):
        raise NotImplementedError
        
    def fax(self, document):
        raise NotImplementedError
        
    def scan(self, document):
        raise NotImplementedError

This interface might seem like a good idea, because if somebody is making  a multifunctional printer, they everything is fine, they need to actually have all of these and they need to implement them. 

In [2]:
class MultiFunctionPrinter(Machine):
    def print(self, document):
        pass
    
    def fax(self, document):
        pass
    
    def scan(self, document):
        pass

Now, the problem is what happens when you want to make, let's say an old fashioned printer, Now, remember that we only have this one interface to work with

In [3]:
class OldFashionedPrinter(Machine):
    def print(self, document):
        # OK
        pass
    
    def fax(self, document):
        # Does not support
        pass
    
    def scan(self, document):
        # Does not support
        pass

Certainly, an old fashioned printer can print, but it connot fax or scan, so what is that we can do here.

- One approach is to simply just do nothing, but this in itself is problematic, Well, because if somebody makes an instance of an old fashioned printer, they're still going to see fax as an interface member. 
- The another alternative is that you start complaining, means you raise a NotImplemented error where you say, for example printer cannot scan this way. 
-----

So, the idea of interface aggregation is basically the following.
- The idea is that instead of having one large interface with several members in it, what you want to do is you want to keep things granular. You want to split this interface into separate parts that people can implement. So if somebody wants to print something, they can have an interface called Printer, if they have scanning functionality, they can implement an additional interafce canned scanner


In [4]:
from abc import ABC, abstractmethod

class Printer(ABC):
    @abstractmethod
    def print(self, document):
        pass
    
class Scanner(ABC):
    @abstractmethod
    def scan(self, document):
        pass
    
class Fax(ABC):
    @abstractmethod
    def fax(self, document):
        pass

Now, for example if you just want a printer, or a photocopier which have both print and scan functionality

In [6]:
class MyPrinter(Printer):
    def print(self, document):
        print("P R I N T E D")
        
ptr = MyPrinter()
ptr.print("doc")

P R I N T E D


In [7]:
class PhotoCopier(Printer, Scanner):
    def print(self, doc):
        print(f"PRINTED: {doc}")
        
    def scan(self, doc):
        print(f"SCANNED: {doc}")
        
pc = PhotoCopier()
pc.print("MRINAL")
pc.scan("SINHA")

PRINTED: MRINAL
SCANNED: SINHA


Now, if you want to have multi functionality printer you can have that

In [8]:
class MultiFunctionDevice(Printer, Scanner, Fax):
    @abstractmethod
    def print(self, doc):
        raise NotImplementedError
        
    @abstractmethod
    def scan(self, doc):
        raise NotImplementedError
        
    @abstractmethod
    def fax(self, doc):
        raise NotImplementedError

Now, if someone want to have a multi function machine, you can say, well this is going to implment multi function machine 

In [9]:
class MultiFunctionMachine(MultiFunctionDevice):
    def __init__(self, printer, scanner, fax):
        self.scanner = scanner
        self.printer = printer
        self.fax = fax
    
    def print(self, doc):
        self.printer.print(doc)
        
    def scan(self, doc):
        self.scanner.scan(doc)
        
    def fax(self, doc):
        self.fax.fax(doc)

## Dependency Inversion Principle